<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/AnalisisDeDatos/1_Indexing/indexing.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>
<div align="center"> Recordá abrir en una nueva pestaña </div>

# Tabla de Contenidos

    I. Análisis de datos con Pandas
    II. Los objetos fundamentales de Pandas
        I. Series
        II. DataFrames
        III. Índices
    III. Exploración
        I. Filtrando un DataFrame (Indexing)
            I. Boolean Indexing
                I. Máscara booleana
                II. Máscara booleana con muchas condiciones
            II. Boolean indexing con query()
            III. Fancy Indexing
        II. Funciones de Agregación
    IV. Otros análisis descriptivos
        I. Para las variables numéricas
        II. Para las variables categóricas
        III. Ordenar por columnas y limitar la cantidad de resultados
    V. Anexo: volviendo al tema de la vectorización

# Análisis de datos con Pandas

Pandas es una de las librerías de Python más usadas para análisis de datos. El nombre pandas viene de "Panel Data Analysis" y su funcionalidad permite hacer operaciones sobre datos que se encuentran en memoria de manera eficiente. 

Pandas es útil para trabajar sobre datos tabulares, con dos condiciones importantes:

I. Los datos se encuentran enteramente en la memoria RAM. Con lo cual, el tamaño de los datos que podemos manipular está limitado por el hardware. Como regla de pulgar, es una buena práctica no ocupar más de 1/3 de la memoria RAM de nuestro dispositivo con el dataset. Así, si estamos trabajando localmente en una notebook con 8GB de memoria RAM no es recomendable procesar datasets de más de 2.33GB.

II. En pandas, las operaciones sobre filas y columnas son, en general, eficientes porque se hacen de forma "vectorizada". En realidad esta optimización, se hace desde numpy, una librería para realizar operaciones matemáticas que se utilizó a su vez para escribir pandas. 

Las operaciones vectorizadas son las que se realizan en bloque en vez de caso por caso. Las computadoras de hoy tienen la capacidad de recibir muchas instrucciones juntas y procesar varias de ellas a la vez. Por ejemplo, si nuestro hardware tiene la capacidad de procesar 4 operaciones juntas, el resultado de vectorizar una operación matemática es el siguiente:

<img src = 'img/vectorizacion.png' /> 


En el primer caso hay que hacer 5 operaciones y en el segundo caso sólo dos.

Es importante entender, entonces, que Pandas trabaja de esta manera y que por eso es una de las herramientas más elegidas para manipular datos en memoria.


In [2]:
# Siempre al principio, importamos las librerías.
import pandas as pd
import numpy as np



# Los objetos fundamentales de Pandas

## Series

Las series son "columnas" que de una tabla que están asociadas a un índice y a un nombre. Igual que una lista común de Python es una secuencia de elementos ordenados, pero a diferencia de la lista está asociada a más información.

In [3]:
# Las series se pueden crear a partir de una lista
serie = pd.Series(['a','b','c'])

In [4]:
# Propiedades importantes de las series
print('Tipo de objetos que tiene ', serie.dtype)
print('Nombre ', serie.name)
print('Index ',serie.index)
print('Valores ',serie.values)


Tipo de objetos que tiene  object
Nombre  None
Index  RangeIndex(start=0, stop=3, step=1)
Valores  ['a' 'b' 'c']


## DataFrames

Los DataFrames son "tablas", compuestas por varias "columnas" o series que comparten todas un mismo índice. En general los DataFrames se crean a partir de leer tablas de archivos (pueden ser en formato json o csv) pero a veces también se crean a partir de listas de diccionarios o de otras maneras. 

Los DataFrames tienen un objeto Index que describe los nombres de columnas y otro objeto Index que describen los nombres de las filas.

In [47]:
# 1. Leemos un dataset público
df = pd.read_csv('http://cdn.buenosaires.gob.ar/datosabiertos/datasets/sueldo-funcionarios/sueldo_funcionarios_2019.csv')

In [48]:
# Propiedades importantes de los dataframes
print('Columnas ', df.columns)
print('Index ', df.index)
print('Dimensiones ',df.shape)


Columnas  Index(['cuil', 'anio', 'mes', 'funcionario_apellido', 'funcionario_nombre',
       'reparticion', 'asignacion_por_cargo_i', 'aguinaldo_ii',
       'total_salario_bruto_i_+_ii', 'observaciones'],
      dtype='object')
Index  RangeIndex(start=0, stop=385, step=1)
Dimensiones  (385, 10)


In [49]:
# Explorar un DataFrame
df.head()

,cuil,anio,mes,funcionario_apellido,funcionario_nombre,reparticion,asignacion_por_cargo_i,aguinaldo_ii,total_salario_bruto_i_+_ii,observaciones
0,20-17692128-6,2019,1,RODRIGUEZ LARRETA,HORACIO ANTONIO,Jefe de Gobierno,197745.80,0.0,197745.80,NaN
1,20-17735449-0,2019,1,SANTILLI,DIEGO CESAR,Vicejefatura de Gobierno,197745.80,0.0,197745.80,NaN
2,27-24483014-0,2019,1,ACUÑA,MARIA SOLEDAD,Ministerio de Educación e Innovación,224516.62,0.0,224516.62,NaN
3,20-13872301-2,2019,1,ASTARLOA,GABRIEL MARIA,Procuración General de la Ciudad de Buenos Aires,224516.62,0.0,224516.62,NaN
4,20-25641207-2,2019,1,AVOGADRO,ENRIQUE LUIS,Ministerio de Cultura,224516.62,0.0,224516.62,NaN


Si queremos extraer una serie del DataFrame, podemos hacerlo de la misma forma en que extraemos un valor de un diccionario.



In [9]:
serie_mes = df['mes']

In [10]:
type(serie_mes)

pandas.core.series.Series

## Índices

Los índices acompañan a las series y a los Data Frames. Son conjuntos ordenados e inmutables de elementos

In [11]:
ind = pd.Index([2, 3, 5, 7, 11])
ind

Int64Index([2, 3, 5, 7, 11], dtype='int64')

In [12]:
ind[1] = 0

TypeError: Index does not support mutable operations

# Exploración

Vamos a analizar datos de una fuente real. Los ingresos de los funcionarios son información pública que se libera anualmente en el <a href='https://data.buenosaires.gob.ar/dataset/sueldo-funcionarios'>portal de datos abiertos</a> de GCBA.  

En general los 4 primeros pasos para analizar un dataset son:
1. Leerlo
2. Consultar cuáles son las columnas
3. Extraer una muestra
4. Verificar cuántos registros tiene

In [15]:
df = pd.read_csv('sueldo_funcionarios_2019.csv')
df.columns

Index(['cuil', 'anio', 'mes', 'funcionario_apellido', 'funcionario_nombre',
       'reparticion', 'asignacion_por_cargo_i', 'aguinaldo_ii',
       'total_salario_bruto_i_+_ii', 'observaciones'],
      dtype='object')

In [16]:
df.sample(5)

,cuil,anio,mes,funcionario_apellido,funcionario_nombre,reparticion,asignacion_por_cargo_i,aguinaldo_ii,total_salario_bruto_i_+_ii,observaciones
371,20-25567686-6,2019,12,BENEGAS,FERNANDO,SECR Innovación y Transformación Digital,249972.86,124986.43,374959.29,NaN
229,20-24941711-5,2019,8,MURA,MARTIN,Ministerio de Economía y Finanzas,263531.98,0.00,263531.98,NaN
186,20-28908968-4,2019,6,COELHO CHICANO,CHRISTIAN,SS Contenidos,226866.41,113433.21,340299.62,NaN
118,27-21759789-2,2019,4,LEGUIZAMON,ISABELLA KARINA,"Consejo de los Derechos de Niñas, Niños y Adol...",226866.41,0.00,226866.41,NaN
34,27-24483014-0,2019,2,ACUÑA,MARIA SOLEDAD,Ministerio de Educación e Innovación,224516.62,0.00,224516.62,NaN


In [17]:
# La propiedad shape nos devuelve una tupla (filas,columnas)
df.shape

(385, 10)

## Filtrando un DataFrame (Indexing)

Hay muchas técnicas para filtrar un DataFrame. Podemos querer filtrar por columnas o por filas, por posición o por nombre. También podemos querer filtrar por condiciones que se cumplen o no. Cuando no queremos filtrar sobre una dimensión (filas o columnas) usamos ":" para seleccionar todo.


<img src='img/indexing.png' style='height:350px' />



### Boolean Indexing

Supongamos que queremos tomar el dataset de funcionarios y quedarnos únicamente con los que pertenecen al Ministerio de Cultura.
Para eso lo que hacemos es indexar al DataFrame por una condición booleana. Eso implica que debemos crear una serie compuesta por valores True y False para aplicarla como índice a las filas.

Los operadores que sirven para evaluar condiciones sobre las series son:


| S  | Descripción   | S  | Descripción   |   |
|----|---------------|----|---------------|---|
| >= | Mayor o Igual | <= | Menor o Igual |   |
| == | Igual         | != | Distinto      |   |
| >  | Mayor         | <  | Menor         |   |

#### Máscara booleana

Veamos lo que pasa cuando le aplicamos a una serie una condición que devuelve un booleano

In [51]:
df['anio'] != 2019

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
355    False
356    False
357    False
358    False
359    False
360    False
361    False
362    False
363    False
364    False
365    False
366    False
367    False
368    False
369    False
370    False
371    False
372    False
373    False
374    False
375    False
376    False
377    False
378    False
379    False
380    False
381    False
382    False
383    False
384    False
Name: anio, Length: 385, dtype: bool

In [52]:
mascara_booleana = df['anio'] != 2019

Nos devuelve una serie de la misma longitud que la original y que contiene sólo valores True o False. 

In [53]:
type(mascara_booleana)

pandas.core.series.Series

In [55]:
mascara_booleana.shape

(385,)

In [56]:
mascara_booleana.dtype

dtype('bool')

Ahora seleccionemos entonces, los registros que corresponden al Ministerio de Cultura.

In [18]:
df_min_cul = df.loc[df['reparticion'] == 'Ministerio de Cultura',:]

In [19]:
# Veamos la cantidad de casos
df_min_cul.shape

(12, 10)

Algo que puede llegar a confundir sobre el Indexing en Pandas es que en algunos casos se puede ser menos explícito a la hora de filtrar. Por ejemplo si ponemos una condición Booleana, pandas asume que el tipo de indexing es loc y que el filtro es sobre las filas y no sobre las columnas:


In [20]:
df_min_cul = df[df['reparticion'] == 'Ministerio de Cultura']

In [21]:
df_min_cul.shape

(12, 10)

Probemos con otra condición.

Ejercicio: Traer todos los sueldos de la segunda mitad del año...

In [22]:
# Solución


#### Máscara booleana con muchas condiciones

Ahora tratemos de filtrar el dataset por dos condiciones: por ejemplo tomar los sueldos de abril de la secretaria de innovación. 
Para eso tenemos que combinar dos máscaras booleanas con una condición.

| S | Descripcion        | S  | Descripcion |   |
|---|--------------------|----|-------------|---|
| & | AND (y)            | \| | OR (o)      |   |
| ^ | XOR (o exclusivo)  | ~  | NOT (no)    |   |



Por ejemplo: seleccionemos los casos donde o bien se haya cobrado aguinaldo o bien el salario total haya sido mayor que 240.000, pero no las dos cosas. 


In [61]:
df[(df['total_salario_bruto_i_+_ii'] > 240000) ^ (df['aguinaldo_ii'] > 0)]

,cuil,anio,mes,funcionario_apellido,funcionario_nombre,reparticion,asignacion_por_cargo_i,aguinaldo_ii,total_salario_bruto_i_+_ii,observaciones
55,20-26781618-3,2019,2,LARRE,PEDRO ANDRES,"SECR Ciencia, Tecnologia e Innovacion",204017.27,34002.88,238020.15,baja 28/2/2019
97,27-24483014-0,2019,4,ACUÑA,MARIA SOLEDAD,Ministerio de Educación e Innovación,249661.60,0.00,249661.60,NaN
98,20-13872301-2,2019,4,ASTARLOA,GABRIEL MARIA,Procuración General de la Ciudad de Buenos Aires,249661.60,0.00,249661.60,NaN
99,20-25641207-2,2019,4,AVOGADRO,ENRIQUE LUIS,Ministerio de Cultura,249661.60,0.00,249661.60,NaN
100,27-13221055-7,2019,4,BOU PEREZ,ANA MARIA,Ministerio de Salud,249661.60,0.00,249661.60,NaN
101,27-13092400-5,2019,4,FREDA,MONICA BEATRIZ,Sindicatura General de la Ciudad de Buenos Aires,249661.60,0.00,249661.60,NaN
102,20-17110752-1,2019,4,MACCHIAVELLI,EDUARDO ALBERTO,Ministerio de Ambiente y Espacio Público,249661.60,0.00,249661.60,NaN
103,20-22293873-3,2019,4,MIGUEL,FELIPE OSCAR,Jefatura de Gabinete de Ministros,249661.60,0.00,249661.60,NaN
104,20-14699669-9,2019,4,MOCCIA,FRANCO,Ministerio de Desarrollo Urbano y Transporte,249661.60,0.00,249661.60,NaN
105,20-24941711-5,2019,4,MURA,MARTIN,Ministerio de Economía y Finanzas,249661.60,0.00,249661.60,NaN


Ahora veamos los sueldos de febrero de la SECR Ciencia, Tecnologia e Innovacion.

In [63]:
df[(df['mes'] == 2) & (df['reparticion'] == 'SECR Ciencia, Tecnologia e Innovacion')]

,cuil,anio,mes,funcionario_apellido,funcionario_nombre,reparticion,asignacion_por_cargo_i,aguinaldo_ii,total_salario_bruto_i_+_ii,observaciones
55,20-26781618-3,2019,2,LARRE,PEDRO ANDRES,"SECR Ciencia, Tecnologia e Innovacion",204017.27,34002.88,238020.15,baja 28/2/2019


### Boolean indexing con query()

La sintaxis que se utiliza para hacer Boolean indexing es un poco repetitiva. Noten que filtrar (aún en su expresión más corta sin loc ni especificar filas o columnas) implica ESCRIBIR DOS VECES el nombre del dataset. Para crear un shortcut, Pandas ofrece la función .query() 



In [66]:
df_cult = df.query('reparticion == "Ministerio de Cultura"')

Ejercicio: Piensen cómo traducir a la sintaxis de query, estas consultas que ya hicimos:

In [69]:
# df_sem2 = df[df['mes'] > 6]
# Solución


In [ ]:
# df[(df['mes'] == 2) & (df['reparticion'] == 'SECR Ciencia, Tecnologia e Innovacion')]


### Fancy Indexing

Ahora vamos a quedarnos con un subconjunto de columnas del DataFrame.

In [23]:
df_view = df.loc[:,['anio','mes']]

In [24]:
df_view.shape

(385, 2)

Existe una forma menos explícita de hacer esta misma operación. Si pasamos una lista al indexing, pandas asume que el tipo de indexing es loc y que el filtro es sobre las columnas y no las filas:

In [25]:
df_view = df[['anio','mes']]

In [26]:
df_view.shape

(385, 2)

Fíjense lo que pasa si tratamos de acceder a filas utilizando una lista de nombres, en este caso [0,1]. 

In [27]:
# Incorrecto
df_view = df[[3,8]]

KeyError: "None of [Int64Index([3, 8], dtype='int64')] are in the [columns]"

Nos da un error porque cuando pasamos únicamente una lista al indexing, pandas asume que queremos un set de columnas y si los nombres no existen, da error. La forma correcta de hacerlo es pasar una lista de índices y explicitar que vamos a indizar con loc y que seleccionamos todas las columnas.

In [28]:
# Correcto
df_view = df.loc[[3,8],:]

In [29]:
df_view

,cuil,anio,mes,funcionario_apellido,funcionario_nombre,reparticion,asignacion_por_cargo_i,aguinaldo_ii,total_salario_bruto_i_+_ii,observaciones
3,20-13872301-2,2019,1,ASTARLOA,GABRIEL MARIA,Procuración General de la Ciudad de Buenos Aires,224516.62,0.0,224516.62,NaN
8,20-22293873-3,2019,1,MIGUEL,FELIPE OSCAR,Jefatura de Gabinete de Ministros,224516.62,0.0,224516.62,NaN


## Funciones de Agregación

Utilizando Pandas podemos aplicar funciones a nivel de columna. Algunas funciones predefinidas son la media, el desvío estándar y la sumatoria, el valor máximo y el mínimo



In [30]:
df['mes'].max()

12

In [31]:
df['asignacion_por_cargo_i'].mean()

234234.3679999999

In [32]:
df['asignacion_por_cargo_i'].std()

35043.16008466176

In [33]:
df['total_salario_bruto_i_+_ii'].sum()

97988834.36000001

Podemos combinar los filtros que vimos antes con las funciones de agregación para responder preguntas cómo ¿Cuál fue en gasto en asignaciones de funcionarios para la Secretaría de Medios 2019? ¿Y para la de Justicia y Seguridad?

In [34]:
df[df['reparticion'] == 'SECR de Medios']['total_salario_bruto_i_+_ii'].sum()

3232402.25

In [35]:
df[df['reparticion'] == 'SECR Justicia y Seguridad']['total_salario_bruto_i_+_ii'].sum()

3029551.7300000004

Ahora respondamos algunas preguntas: ¿Quién o quiénes del dataset cobran el salario más alto? ¿Y el más bajo?

In [36]:
df[df['total_salario_bruto_i_+_ii'] == df['total_salario_bruto_i_+_ii'].max()]

,cuil,anio,mes,funcionario_apellido,funcionario_nombre,reparticion,asignacion_por_cargo_i,aguinaldo_ii,total_salario_bruto_i_+_ii,observaciones
374,20-13872301-2,2019,12,ASTARLOA,GABRIEL MARIA,Procuración General de la Ciudad de Buenos Aires,275089.75,170855.56,445945.31,NaN


In [37]:
df[df['total_salario_bruto_i_+_ii'] == df['total_salario_bruto_i_+_ii'].min()]

,cuil,anio,mes,funcionario_apellido,funcionario_nombre,reparticion,asignacion_por_cargo_i,aguinaldo_ii,total_salario_bruto_i_+_ii,observaciones
344,27-30744939-6,2019,12,FERRERO,GENOVEVA,SECR Administración de Seguridad y Emergencias,74991.86,110404.68,185396.54,baja al 9/12
348,20-22709722-2,2019,12,LOPEZ,MATIAS,SECR Desarrollo Ciudadano,74991.86,110404.68,185396.54,baja al 9/12
354,20-20008464-1,2019,12,GIUSTI,JOSE LUIS,UPE Transferencia de Funciones y Facultades en...,74991.86,110404.68,185396.54,baja al 9/12
358,20-31164337-2,2019,12,DI BENEDETTO,FEDERICO,SS Comunicacion,74991.86,110404.68,185396.54,baja al 9/12
360,20-28908968-4,2019,12,COELHO CHICANO,CHRISTIAN,SS Contenidos,74991.86,110404.68,185396.54,baja al 9/12
362,20-24424714-9,2019,12,D'ALESSANDRO,MARCELO SILVIO,SECR Justicia y Seguridad,74991.86,110404.68,185396.54,baja al 9/12


# Otros análisis descriptivos

Pandas viene con algunas funciones built-in para ayudar al análisis descriptivo.

## Para las variables numéricas

In [38]:
df.describe()

,anio,mes,asignacion_por_cargo_i,aguinaldo_ii,total_salario_bruto_i_+_ii
count,385.0,385.000000,385.000000,385.000000,385.000000
mean,2019.0,6.631169,234234.368000,20282.084883,254516.452883
std,0.0,3.539077,35043.160085,45248.840725,51434.987870
min,2019.0,1.000000,74991.860000,0.000000,185396.540000
25%,2019.0,4.000000,224516.620000,0.000000,224516.620000
50%,2019.0,7.000000,239470.360000,0.000000,245811.620000
75%,2019.0,10.000000,249972.870000,0.000000,263531.980000
max,2019.0,12.000000,275089.750000,170855.560000,445945.310000


## Para las variables categóricas


In [39]:
df['reparticion'].value_counts()

SECR Justicia y Seguridad                                         13
SECR de Medios                                                    13
SECR Desarrollo Ciudadano                                         13
Ministerio de Salud                                               13
Ministerio de Cultura                                             12
Consejo de los Derechos de Niñas, Niños y Adoles - Presidencia    12
Ministerio de Gobierno                                            12
SECR Legal y Técnica                                              12
Sindicatura General de la Ciudad de Buenos Aires                  12
SECR Integración Social Para Personas Mayores                     12
SECR Planificación, Evaluación y Coordinación de Gestión          12
Ente de Turismo Ley Nº 2627                                       12
Jefe de Gobierno                                                  12
Jefatura de Gabinete de Ministros                                 12
Procuración General de la Ciudad d

## Ordenar por columnas y limitar la cantidad de resultados

Otra forma de resolver el problema de encontrar el mayor y el menos es con el método sort_values. Este método puede recibir un valor único (nombre de columna) o una lista (con varias columnas) y un orden asc o desc. Por default el orden es asc.

Si combinamos el ordenamiento con el método head() para limitar la cantidad de resultados, podemos encontrar los N primeros. 

In [40]:
# Recordemos cómo abrir la documentación de un método
df.sort_values?

In [41]:
# Calculamos el máximo
df.sort_values('total_salario_bruto_i_+_ii',ascending=False).head(1)

,cuil,anio,mes,funcionario_apellido,funcionario_nombre,reparticion,asignacion_por_cargo_i,aguinaldo_ii,total_salario_bruto_i_+_ii,observaciones
374,20-13872301-2,2019,12,ASTARLOA,GABRIEL MARIA,Procuración General de la Ciudad de Buenos Aires,275089.75,170855.56,445945.31,NaN


In [42]:
# Calculamos el mínimo
df.sort_values('total_salario_bruto_i_+_ii').head(1)

,cuil,anio,mes,funcionario_apellido,funcionario_nombre,reparticion,asignacion_por_cargo_i,aguinaldo_ii,total_salario_bruto_i_+_ii,observaciones
344,27-30744939-6,2019,12,FERRERO,GENOVEVA,SECR Administración de Seguridad y Emergencias,74991.86,110404.68,185396.54,baja al 9/12


# Anexo: volviendo al tema de la vectorización

¿Por qué es tan importante trabajar con Pandas y no con funciones escritas por nosotros en Python nativo y que procesen los datos dentro de un for loop?

Por un lado está la comodidad. Hay mucha funcionalidad que ya está desarrollada en Pandas. Existen funciones que resuelven muchos de los problemas clásicos de manipular datos: agrupar, sumarizar, sacar estadísticas, filtrar, etc. Pero además hay una razón de performance. 

Veamos una demostración de que vectorizar es más eficiente. Vamos a crear dos listas de 1.000.000 de números aleatorios cada una y vamos a tratar de multiplicar elemento por elemento con pandas y sin pandas:



In [43]:
lista1 = list(np.random.randint(1, 100, 1000000))
lista2 = list(np.random.randint(1, 100, 1000000))

In [44]:
%%timeit 
for x,y in zip(lista1,lista2):
    x * y

71.7 ms ± 346 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Ahora probemos hacer lo mismo con dos series de Pandas

In [45]:
serie1 = pd.Series(lista1)
serie2 = pd.Series(lista2)

In [46]:
%%timeit 
resultado = serie1 * serie2

953 µs ± 3.11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Conclusión: la operación vectorizada es <strong> más de 70 veces más rápida.</strong>